### DSPT6 - Unit 3 Module 2 - Consuming Data from an API

The purpose of this notebook is to demonstrate:
- Connect to the Twitter API (and twitter_scraper) to query for tweets and user information by various parameters
- Convert tweet text using SpaCy into numerical embeddings that can be use in a predictive model

In [1]:
import tweepy

In [2]:
## ADD YOUR CREDENTIALS HERE
TWITTER_CONSUMER_API_KEY = 'u21oPsosPqUKamwd0gwBO6zf1'
TWITTER_CONSUMER_API_SECRET = 'Oa8WNzHfDlfFl2gUbEzaJY3qicN3a3O5P3YYvBzzCH4BLRFWuf'
TWITTER_ACCESS_TOKEN = '1259857629200908288-Qg2LxIvvw5W9O5eRPMQhBAE80Gutc5'
TWITTER_ACCESS_TOKEN_SECRET = 'GJHPQht7k54NBgzsP1mGZAz5GLsER9EghXSe2VunggcYb'

In [3]:
TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [6]:
twitter_user = TWITTER.get_user('elonmusk')

In [8]:
twitter_user.followers_count

39058219

In [4]:
user = TWITTER.get_user('dustinstri92')

In [5]:
tweet = user.timeline()[0]

In [17]:
def add_or_update_user(username, nlp):
    try:
        twitter_user = TWITTER.get_user(username)
        tweets = twitter_user.timeline(count=200,
                                    exclude_replies=True,
                                    include_rts=False,
                                    tweet_mode='extended')
        embeddings = vectorize_tweet(nlp, tweets[0].full_text)
    except Exception as e:
        print('Error processing {}: {}'.format(username, e))
    return tweets, embeddings

In [18]:
def add_user_tweepy(username):
    '''Add a user and their tweets to database'''
    try:
        # Get user info from tweepy
        twitter_user = TWITTER.get_user(username)
        # Add to User table (or check if existing)
        db_user = (User.query.get(twitter_user.id) or
                   User(id=twitter_user.id,
                        username=username,
                        follower_count=twitter_user.followers_count))
        db.session.add(db_user)
        # Get tweets ignoring re-tweets and replies
        tweets = twitter_user.timeline(count=200,
                                       exclude_replies=True,
                                       include_rts=False,
                                       tweet_mode='extended',
                                       since_id=db_user.newest_tweet_id)
        # Add newest_tweet_id to the User table
        if tweets:
            db_user.newest_tweet_id = tweets[0].id
        # Loop over tweets, get embedding and add to Tweet table
        for tweet in tweets:
            # Get an examble basilica embedding for first tweet
            embedding = BASILICA.embed_sentence(tweet.full_text, model='twitter')
            # Add tweet info to Tweet table
            db_tweet = Tweet(id=tweet.id,
                             text=tweet.full_text[:300],
                             embedding=embedding)
            db_user.tweet.append(db_tweet)
            db.session.add(db_tweet)
    except Exception as e:
        print('Error processing {}: {}'.format(username, e))
        raise e
    else:
        # If no errors happend than commit the records
        db.session.commit()

### SpaCy Embeddings

In [7]:
import spacy

In [8]:
nlp = spacy.load('en_core_web_sm')

In [10]:
doc = nlp(tweet.text)

In [16]:
print([token.text for token in doc])

[&#39;To&#39;, &#39;err&#39;, &#39;is&#39;, &#39;human&#39;, &#39;,&#39;, &#39;but&#39;, &#39;to&#39;, &#39;really&#39;, &#39;foul&#39;, &#39;things&#39;, &#39;up&#39;, &#39;requires&#39;, &#39;the&#39;, &#39;root&#39;, &#39;password&#39;, &#39;.&#39;, &#39;\n&#39;, &#39;-Unknown&#39;]


### Bringing it all together